## Prompt Templates, Few-Shot Learning & Output Parsing

This notebook demonstrates how to use LangChain's Prompt Templates, few-shot prompting techniques, and structured output parsing with a local open-source language model.

We will use the instruction-tuned model `"NousResearch/Nous-Hermes-2-Mistral-7B-DPO"` throughout, and explore:
- Prompt templates for reusability and clarity
- Few-shot prompting to guide the model with examples
- Structured output parsing using Pydantic


### Prompt Templates in LangChain

LangChain’s `PromptTemplate` lets you define reusable prompt structures with placeholders for dynamic input.

```python
from langchain.prompts import PromptTemplate

prompt = PromptTemplate(
    input_variables=["topic"],
    template="Explain the following topic in simple terms:

{topic}"
)

print(prompt.format(topic="What is machine learning?"))
```

This is helpful for keeping prompts clean and consistent across inputs.


In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_huggingface.llms import HuggingFacePipeline
from langchain_huggingface import ChatHuggingFace
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import SystemMessagePromptTemplate, HumanMessagePromptTemplate, ChatPromptTemplate, AIMessagePromptTemplate
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel
from typing import List
import json
import re
import os

In [2]:
# path_to_model = "/gpfs/data/fs70824/LLMs_models_datasets/models" # on VSC5
path_to_model = "/leonardo_scratch/fast/EUHPC_D20_063/huggingface/models/Nous-Hermes-2-Mistral-7B-DPO" # on Leonardo

In [3]:
# model_id = "NousResearch/Nous-Hermes-2-Mistral-7B-DPO" # on VSC5

tokenizer = AutoTokenizer.from_pretrained(path_to_model)
model = AutoModelForCausalLM.from_pretrained(path_to_model, local_files_only=True, device_map="auto")

text_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=False,
    max_new_tokens=512,
    do_sample=True,
    temperature=0.7,
)

llm = HuggingFacePipeline(pipeline=text_pipeline)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cuda:0


In [4]:
prompt_template = PromptTemplate(
    input_variables=["topic"],
    template="Explain the following topic in simple terms:\n\n{topic}"
)

print(prompt_template.format(topic="What is machine learning?"))

Explain the following topic in simple terms:

What is machine learning?


In [5]:
response = llm.invoke(prompt_template.format(topic="What is machine learning?"))
print(response)



Machine learning is a subset of artificial intelligence. It involves training a computer or software program to identify patterns in data, and then apply these patterns to new data, making predictions or decisions on this new data. Essentially, machine learning allows a computer to learn from data, without being explicitly programmed.

For example, machine learning can be used to create a system that can identify fraudulent credit card transactions. By training the system on a set of historical data that includes both fraudulent and non-fraudulent transactions, the system can learn to recognize the patterns that are typical of fraudulent transactions. Once the system has learned these patterns, it can then apply them to new data, quickly flagging any new transactions that match these patterns as potentially fraudulent, enabling the credit card company to take appropriate action.


<br>
Let's have a look at another example:

In [6]:
simplify_prompt = PromptTemplate(
    input_variables=["clause"],
    template="""
You are a legal assistant that simplifies complex legal clauses into plain, understandable English.

Clause:
{clause}

Simplified Explanation:
"""
)

In [7]:
legal_clause = (
    "The lessee shall indemnify and hold harmless the lessor from any liabilities, damages, "
    "or claims arising out of the use of the premises, except in cases of gross negligence."
)

formatted_prompt = simplify_prompt.format(clause=legal_clause)
response = llm.invoke(formatted_prompt)

print("Simplified:\n", response)

Simplified:
 The person renting the property (lessee) promises to cover and protect the person or company owning the property (lessor) from any problems or costs that may come up due to someone using the property. However, this does not apply if the problem is caused by a serious lack of care or attention.


Let's try the same thing with OpenAI's API:

In [ ]:
import os, getpass
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API key (input is hidden): ")

In [ ]:
from langchain_openai import ChatOpenAI

# Create an LLM that talks to OpenAI (reads OPENAI_API_KEY from env)
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.7,     # like HF's temperature
    max_tokens=512       # analogous to HF's max_new_tokens
)

In [ ]:
response = llm.invoke(formatted_prompt)

print("Simplified:\n", response)

### Few Shot Prompt Template

Let's go over an example where you want a historical conversation to show the LLM Chat Bot a few examples, known as "Few Shot Prompts". We essentially provide some examples *before* sending the message history to the LLM. Be careful not to make the entire message too long, as you may hit context limits (but the latest models have quite large contexsts.

LangChain distinguishes between:

 - PromptTemplates for simple string prompts
 - MessagePromptTemplates for structured chat-style prompts using roles like system, user, assistant

So SystemMessagePromptTemplate helps build structured prompts that work with ChatModels

**Creating Example Inputs and Outputs**

In [8]:
template = "You are a helpful assistant that translates complex legal terms into plain and understandable language."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)

In [9]:
legal_text_1 = "Notwithstanding any provision to the contrary herein, the indemnitor agrees to indemnify, defend, and hold harmless the indemnitee from and against any and all claims, liabilities, damages, or expenses (including, without limitation, reasonable attorney’s fees) arising out of or related to the indemnitor’s acts or omissions, except to the extent that such claims, liabilities, damages, or expenses result from the gross negligence or willful misconduct of the indemnitee."
example_input_1 = HumanMessagePromptTemplate.from_template(legal_text_1)

plain_text_1 = "One party agrees to cover any costs, claims, or damages that happen because of their actions, including legal fees. However, they do not have to pay if the other party was extremely careless or acted intentionally wrong."
example_output_1 = AIMessagePromptTemplate.from_template(plain_text_1)

legal_text_2 = "This agreement shall be binding upon and inure to the benefit of the parties hereto and their respective heirs, executors, administrators, successors, and assigns, and shall not be assignable by either party without the prior written consent of the other, except that either party may assign its rights and obligations hereunder in connection with a merger, consolidation, or sale of substantially all of its assets."
example_input_2 = HumanMessagePromptTemplate.from_template(legal_text_2)

plain_text_2 = "This agreement applies to both parties and their future representatives, such as heirs or business successors. Neither party can transfer their rights under this agreement to someone else unless they get written permission. However, if one party merges with another company or sells most of its assets, they can transfer their rights without permission."
example_output_2 = AIMessagePromptTemplate.from_template(plain_text_2)

In [10]:
human_template = "{legal_text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

In [11]:
chat_prompt = ChatPromptTemplate.from_messages(
    [system_message_prompt, example_input_1, example_output_1, example_input_2, example_output_2, human_message_prompt]
)

In [12]:
some_example_text = "Any waiver of any term or condition of this agreement shall not be deemed a continuing waiver of such term or condition, nor shall it be considered a waiver of any other term or condition hereof. No failure or delay by either party in exercising any right, power, or privilege under this agreement shall operate as a waiver thereof, nor shall any single or partial exercise preclude any other or further exercise thereof or the exercise of any other right, power, or privilege."
request = chat_prompt.format_prompt(legal_text=some_example_text).to_messages()

In [13]:
result = llm.invoke(request)

In [14]:
print(result)


AI: If one side lets something go without enforcing it, that doesn't mean they give up the right to enforce it in the future. Also, one action does not prevent them from taking further action or enforcing other parts of the agreement.


### Parsing output

Large language models (LLMs) typically generate free-form text, which is great for human conversation — but not ideal when we want to **extract specific information** or **automate downstream tasks**.

---

**The Problem**
Imagine asking an LLM:

> "Summarize this contract and give me the parties involved, the start date, and any penalties."

If the model responds with a long paragraph, it becomes difficult to:
- Reliably extract the pieces you need
- Validate whether the answer is complete
- Feed the output into another system

---

**The Solution**
Structured Output (e.g. JSON)
By instructing the LLM to return data in a structured format like JSON, we can:
- Parse the output automatically, although this does not always work
- Validate that required fields are present
- Integrate with other tools and code seamlessly

---

Structured output turns the LLM into a more reliable component of your application.  
Parsing with tools like Pydantic ensures your data is clean, complete, and ready for automation.


**Define format**
Let's first see if we can get the output in form of a JSON object, by adding that request to the system prompt:

In [15]:
template = "You are a helpful assistant that translates complex legal terms into plain and understandable language.  Respond only with a JSON object containing a single key 'translation' and its corresponding value."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)

In [16]:
chat_prompt = ChatPromptTemplate.from_messages(
    [system_message_prompt, example_input_1, example_output_1, example_input_2, example_output_2, human_message_prompt]
)

In [17]:
some_example_text = "Any waiver of any term or condition of this agreement shall not be deemed a continuing waiver of such term or condition, nor shall it be considered a waiver of any other term or condition hereof. No failure or delay by either party in exercising any right, power, or privilege under this agreement shall operate as a waiver thereof, nor shall any single or partial exercise preclude any other or further exercise thereof or the exercise of any other right, power, or privilege."
request = chat_prompt.format_prompt(legal_text=some_example_text).to_messages()

In [18]:
result = llm.invoke(request)

In [19]:
result

"\nAI: If any part of this agreement is ignored or forgiven, it doesn't mean other parts can be ignored too. Also, not using a right, power, or privilege at one time doesn't stop it from being used in the future, and using it once doesn't prevent using it again or using other rights, powers, or privileges."

That clearly didn't do the trick.

#### Pydantic

[Pydantic](https://docs.pydantic.dev/) is a Python library for defining data models with validation. With LangChain, it allows you to:
- Define the structure you expect from the model
- Automatically parse the raw LLM output
- Catch errors if fields are missing or malformed

---

**Example**

1. Define a Pydantic model

```python
from pydantic import BaseModel
from typing import List

class ClauseSummary(BaseModel):
    parties: List[str]
    start_date: str
    penalty_clause: str
```

This defines the structure we want the LLM to return — a JSON object with:
- A list of `parties`
- A `start_date`
- A `penalty_clause` string

---

2. Set up a parser using LangChain

```python
from langchain.output_parsers import PydanticOutputParser

parser = PydanticOutputParser(pydantic_object=ClauseSummary)
```

This parser will take a raw string (from the LLM) and try to convert it into a `ClauseSummary` object.

---

3. Include the schema in the system prompt

```python
format_instructions = parser.get_format_instructions()

prompt = PromptTemplate(
    input_variables=["clause", "format_instructions"],
    template="""
Extract the following fields from the contract clause below and return them in **valid JSON format ONLY**, with no extra text or explanation.

Clause:
{clause}

{format_instructions}
"""
)
```

> The `format_instructions` tells the LLM exactly what JSON structure to return, based on your Pydantic model.

---

4. Run the LLM and parse the output

```python
response = llm.invoke(prompt)

try:
    parsed = parser.parse(response.content)
    print(parsed.dict())
except Exception as e:
    print("Could not parse output.")
    print("Raw response:", response.content)
    print(e)
```

If the model returns a correctly structured JSON string, you now get a real Python object with attributes you can use:
```python
parsed.parties
parsed.start_date
parsed.penalty_clause
```

---

With this model, you can ensure the LLM responds in a way that fits your expected format — or fail gracefully when it doesn't.<br>
Let's try out this example:

In [20]:
# Define output structure
class ClauseSummary(BaseModel):
    parties: List[str]
    start_date: str
    penalty_clause: str

In [21]:
# Set up parser
parser = PydanticOutputParser(pydantic_object=ClauseSummary)
format_instructions = parser.get_format_instructions()

In [22]:
# Create prompt with correct input variables
prompt = PromptTemplate(
    input_variables=["clause", "format_instructions"],
    template="""
Extract the following fields from the contract clause below and return them in **valid JSON format ONLY**, with no extra text or explanation.

Clause:
{clause}

{format_instructions}
"""
)

In [23]:
# Clause to parse
clause_text = (
    "The agreement between Acme Corp and Beta LLC begins on January 1, 2025. "
    "If either party breaks the agreement, a €5,000 penalty applies."
)

In [24]:
# Format the full prompt
full_prompt = prompt.format(clause=clause_text, format_instructions=format_instructions)

# Run the model
response = llm.invoke(full_prompt)

In [25]:
# Parse the response
try:
    parsed = parser.parse(response)
    print(parsed.dict())
except Exception as e:
    print("Could not parse output.")
    print("Raw response:", response)
    print(e)

{'parties': ['Acme Corp', 'Beta LLC'], 'start_date': 'January 1, 2025', 'penalty_clause': '€5,000 penalty'}


/tmp/ipykernel_984514/1447590507.py:4: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  print(parsed.dict())


In [26]:
print(parsed.parties)
print(parsed.start_date)
print(parsed.penalty_clause)

['Acme Corp', 'Beta LLC']
January 1, 2025
€5,000 penalty


<br>
Let's try that on the example which tried to simplify legal clauses and output them in the JASON format.

In [27]:
# Define output schema with Pydantic 
class LegalSimplification(BaseModel):
    translation: str

parser = PydanticOutputParser(pydantic_object=LegalSimplification)

In [28]:
# Define the system prompt
format_instructions = parser.get_format_instructions()

system_message = SystemMessage(content=(f"""
    "You are a helpful assistant that translates complex legal terms into plain and understandable language. "
    "Respond only with in this format {format_instructions}"
    "Do not ask for clarification. Always use the given legal input."
    """
))

In [29]:
# Define few-shot examples
examples = [
    {
        "input": "Notwithstanding any provision to the contrary herein, the indemnitor agrees to indemnify, defend, and hold harmless the indemnitee...",
        "output": "One party agrees to cover any costs, claims, or damages that happen because of their actions..."
    },
    {
        "input": "This agreement shall be binding upon and inure to the benefit of the parties...",
        "output": "This agreement applies to both parties and their future representatives..."
    }
]

few_shot_messages = []
for ex in examples:
    few_shot_messages.append(HumanMessage(content=ex["input"]))
    few_shot_messages.append(AIMessage(content=f'{{"translation": "{ex["output"]}"}}'))

In [30]:
# Define the legal input text
legal_text = (
    "Any waiver of any term or condition of this agreement shall not be deemed a continuing waiver of such term "
    "or condition, nor shall it be considered a waiver of any other term or condition hereof. No failure or delay "
    "by either party in exercising any right, power, or privilege under this agreement shall operate as a waiver "
    "thereof, nor shall any single or partial exercise preclude any other or further exercise thereof or the "
    "exercise of any other right, power, or privilege."
)

user_message = HumanMessage(content=legal_text)

In [31]:
# Build full message list
messages = [system_message] + few_shot_messages + [user_message]

# Sanity check the prompt
print("\n\n===== Prompt Sent to Model =====")
for m in messages:
    print(f"{m.type.upper()}: {m.content}\n")



===== Prompt Sent to Model =====
SYSTEM: 
    "You are a helpful assistant that translates complex legal terms into plain and understandable language. "
    "Respond only with in this format The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"translation": {"title": "Translation", "type": "string"}}, "required": ["translation"]}
```"
    "Do not ask for clarification. Always use the given legal input."
    

HUMAN: Notwithstanding any provision to the contrary herein, the indemnitor agrees to indemnify, defend, and hold harmless the indemnitee...

AI: {"translation": "One par

In [32]:
# Create HF text-generation pipeline (in this case with sampling for creativity)
hf_pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
    return_full_text=False
)

Device set to use cuda:0


In [33]:
# Wrap in LangChain-compatible LLM
wrapped_llm = HuggingFacePipeline(pipeline=hf_pipe)
llm = ChatHuggingFace(llm=wrapped_llm)

In [34]:
# Generate model output
raw_output = llm.invoke(messages)

In [35]:
print(raw_output.content)

{"translation": "If one side gives up a part of this agreement, it doesn't mean they give up the whole thing or any other part. If one side doesn't use their rights, it doesn't mean they can't use them later or use other rights."}


In [36]:
# Extract valid JSON from output

def extract_first_json(text):
    match = re.search(r'\{.*?\}', text, re.DOTALL)
    return match.group(0) if match else text.strip()

try:
    output_text = raw_output.content if isinstance(raw_output, AIMessage) else raw_output
    clean_output = extract_first_json(output_text)
    result = parser.parse(clean_output)

    print("\nSimplified translation:")
    print(result.translation)

    # Combine original and simplified
    entry = {
        "legal_text": legal_text,
        "translation": result.translation
    }

    # Load existing data if file exists
    data = []
    output_file = "simplified_output.json"
    if os.path.exists(output_file):
        with open(output_file, "r") as f:
            try:
                data = json.load(f)
                if not isinstance(data, list):
                    print("Warning: existing file is not a list. Overwriting.")
                    data = []
            except json.JSONDecodeError:
                data = []

    # Append new entry
    data.append(entry)

    # Write back to file
    with open(output_file, "w") as f:
        json.dump(data, f, indent=2)

    print(f"\nAppended to {output_file}")

except Exception as e:
    print("\nCould not parse output.")
    print("Raw output:", raw_output)
    print(e)


Simplified translation:
If one side gives up a part of this agreement, it doesn't mean they give up the whole thing or any other part. If one side doesn't use their rights, it doesn't mean they can't use them later or use other rights.

Appended to simplified_output.json
